In [1]:
# PyTorch
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

# For data preprocess
import numpy as np
import pandas as pd
from pandas import DataFrame


# For plotting
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
plt.rcParams['font.sans-serif']=['SimHei'] #pyplot中文显示



In [2]:
class RNN (nn.Module):
    def __init__(self):
        super(RNN,self).__init__() 
        self.lstm = nn.LSTM(20,8,2) 
        self.out = nn.Linear(8,20) 
        self.dropout = nn.Dropout(p=0.3)
    def forward(self,x):
        x1,_ = self.lstm(x)
        a,b,c = x1.shape
        out = self.out(x1.view(-1,c))#将lstm输出的三维数据x1调整成二维数据，最后的特征维度不能变
        out = self.dropout(out)
        out1 = out.view(a,b,-1) #把二维的out调整成三维数据，下一次循环使用
        return out1
rnn = RNN()

#参数寻优，计算损失函数
optimizer = torch.optim.Adam(rnn.parameters(),lr = 0.02)
loss_func = nn.MSELoss()

In [3]:
def data_processing(train_data,ans_data0,keyword):
    time_stamp=20 #十分钟的数据，题目本来要求是利用一小时的数据，但如果用了1h(time_stamp=120),训练效果emmmmmm
    train = pd.read_csv(train_data)  #训练集
    #'C:/Users/62331/Desktop/ML/机组数据驱动的风电场短期风况预测/风场1/x26/2018-07-30.csv'
    train=train.dropna()
    #train=train[['变频器电网侧有功功率', '外界温度', '风向', '风速']]
    train=train[[keyword]]
    x_train= []
    y_train= []
    for i in range(time_stamp, len(train)-21):
        x_train.append(train[i-time_stamp:i])
        y_train.append(train[i:i+20])
    x_train = np.array(x_train)
    y_train = np.array(y_train)
    x_train = x_train.reshape(-1, 1, 20) #将训练数据调整成pytorch中lstm算法的输入维度
    y_train = y_train.reshape(-1, 1, 20)  #将目标值调整成pytorch中lstm算法的输出维度
    x_train = torch.from_numpy(x_train)
    y_train = torch.from_numpy(y_train)
    print("read finished,start train")

    for i in range(100):
        var_x = Variable(x_train).type(torch.FloatTensor)
        var_y = Variable(y_train).type(torch.FloatTensor)
        out = rnn(var_x)
        loss = loss_func(out,var_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #if (i+1)%100==0:
        #    print('Epoch:{}, Loss:{:.5f}'.format(i+1, loss.item()))
            
    
    
    
    
    
    print("train finished")
    #预测        
    for i in range(1,10):
        ans_data=ans_data0+"_0"+str(i)+".csv"
        ans = pd.read_csv(ans_data) 
        ans = ans[[keyword]]
        x_ans= []
        x_ans.append(ans[-time_stamp-1:-1])
        x_ans = np.array(x_ans)
        x_ans = x_ans.reshape(-1, 1, 20) #将训练数据调整成pytorch中lstm算法的输入维度
        x_ans = torch.from_numpy(x_ans)
        x_ans = Variable(x_ans).type(torch.FloatTensor)
        pred = rnn(x_ans)
        pred_test = pred.view(-1).data.numpy()  #转换成一维的ndarray数据，这是预测值
        
        outpath=ans_data[:15]+"ans/"+keyword+"/"+ans_data[22:26]+ans_data[26:29]+ans_data[-8:]
        df = pd.DataFrame(pred_test,columns=[keyword])
        df.to_csv(outpath,index=False,header=True)
        
    print("Finished writing to ",outpath)
        
        
        
        
    for i in range(10,21):
        ans_data=ans_data0+"_"+str(i)+".csv"
        ans = pd.read_csv(ans_data) 
        ans = ans[[keyword]]
        x_ans= []
        x_ans.append(ans[-time_stamp-1:-1])
        x_ans = np.array(x_ans)
        x_ans = x_ans.reshape(-1, 1, 20) #将训练数据调整成pytorch中lstm算法的输入维度
        x_ans = torch.from_numpy(x_ans)
        x_ans = Variable(x_ans).type(torch.FloatTensor)
        pred = rnn(x_ans)
        pred_test = pred.view(-1).data.numpy()  #转换成一维的ndarray数据，这是预测值
        
        outpath=ans_data[:15]+"ans/"+keyword+"/"+ans_data[22:26]+ans_data[26:29]+ans_data[-8:]
        df = pd.DataFrame(pred_test,columns=[keyword])
        df.to_csv(outpath,index=False,header=True)
        
    print("Finished writing to ",outpath)
        

In [5]:
Path_data='D:/ML/HFenergy/风场1/x'
Path_test='D:/ML/HFenergy/测试集_初赛/风场1/x'
for i in range (26,51):
    train=Path_data+str(i)+'/2018-08-01.csv'
    test=Path_test+str(i)+'/秋'
    data_processing(train,test,'风向')
    data_processing(train,test,'风速')



read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x26秋_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x26秋_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x26秋_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x26秋_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x27秋_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x27秋_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x27秋_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x27秋_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x28秋_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场1/x28秋_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x28秋_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场1/x28秋_20.csv
read finished,start train
train finished
Finis

In [7]:
location='风场2'
Path_data='D:/ML/HFenergy/风场2/x'
Path_test='D:/ML/HFenergy/测试集_初赛/风场2/x'
for i in range (25,50):
    train=Path_data+str(i)+'/2017-02-01.csv'
    test=Path_test+str(i)+'/春'
    data_processing(train,test,'风向')  
    data_processing(train,test,'风速')

read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x25春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x25春_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x25春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x25春_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x26春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x26春_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x26春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x26春_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x27春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风向/风场2/x27春_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x27春_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x27春_20.csv
read finished,start train
train finished
Finis

In [11]:
Path_data='D:/ML/HFenergy/风场2/x'
Path_test='D:/ML/HFenergy/测试集_初赛/风场2/x'
for i in range (40,45):
    train=Path_data+str(i)+'/2018-07-01.csv'
    test=Path_test+str(i)+'/秋'
    #data_processing(train,test,'风向')
    data_processing(train,test,'风速')

read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x40秋_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x40秋_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x41秋_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x41秋_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x42秋_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x42秋_20.csv
read finished,start train
train finished
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x43秋_09.csv
Finished writing to  D:/ML/HFenergy/ans/风速/风场2/x43秋_20.csv
read finished,start train
train finished


FileNotFoundError: [Errno 2] No such file or directory: 'D:/ML/HFenergy/测试集_初赛/风场2/x44/秋_09.csv'